# PROYECTO EDA

## Fútbol de posesión, o no!

En este trabajo, he tratado de encontrar la influencia de la posesión de balón en el fútbol español para las últimas tres temporadas. Para ello he utilizado solamente datos de la web fbref.com que aporta documentación gratuita de la mayoría de ligas del mundo. Para descargar los datos he utilizado el web scraping con html, para volcarlos todos a un notebook, ya que la página tiene muchas tablas.  

In [ ]:
df_1 = pd.read_html("https://fbref.com/es/comps/12/2020-2021/Estadisticas-2020-2021-La-Liga")                               

df_1_0 = df_1[0]

Una vez descargadas todas las tablas que necesito, he limpiado los dataframe y eliminado las columnas que no me hacían falta para mi análisis. Además, 
he creado dos columnas nuevas. El siguiente paso ha sido concatenar todas las tablas y visualizarlas, ordenandolas por clasificación en un principio y más adelante por la columna que fuese necesario.
He creado una tabla descriptica y una de cardinalidad aunque esta última no me ha servido demasiado(algo he hecho mal).
Para continuar, he sacado la media, mediana, algunos percentiles, el índice intercuantílico y los rangos de cada uno de los dataframe(3). También el coeficiente de variabilidad.


In [ ]:
df_liga20.describe().loc['mean']                                                                                         

df_liga20.describe().loc['50%']                                                                                               

print("Percentil mínimo: ", df_liga20['Posesion'].min())
print("Percentil 10: ", df_liga20['Posesion'].quantile(0.1))
print("Percentil 50: ", df_liga20['Posesion'].quantile(0.5))
print("Percentil 90: ", df_liga20['Posesion'].quantile(0.9))
print("Percentil 95: ", df_liga20['Posesion'].quantile(0.95))
print("Percentil máximo: ", df_liga20['Posesion'].max())                                                                          

def get_IQR(df, col):
    return df[col].quantile(0.75) - df[col].quantile(0.25)

He hecho un describe() de los dataframe y despues he comenzado con las visualizaciones de las variables, comenzado de manera individual por las que he creído mas inportantes.                                                                                                 

In [ ]:
plot_combined_graphs(df_liga21, ['Posesion'], whisker_width = 4.5);

Luego he seguido combinando variables según los datos me iban dando información

In [ ]:
plot_combined_graphs(df_liga20, ['GF', 'GC'], whisker_width = 4.5);

Tambien he creado unos gráficos de densidad comprando variables para comprobar que la hipotesis establecida iba en lo cierto

In [ ]:
sns.kdeplot(data=df_liga22, x=df_liga22['Posesion'], fill=True, label=df_liga22['Posesion']) 
sns.kdeplot(data=df_liga22, x=df_liga22['Pts'], fill=True, label=df_liga22['Pts'])  

plt.xlabel('Posesión vs Puntos')
plt.ylabel('Densidad')
plt.title('Gráfico de densidad')

# Muestra el gráfico
plt.show()

He creado unos gráficos de barras horizontales porque creo que la visualización lateral con tantos valores(20) se apreciaba mejor.

In [ ]:
plt.figure(figsize=(15,9))
plt.barh(df_liga22.Equipo, df_liga22.Posesion)
plt.title('Valor de la posesion por equipo', fontsize=15)
plt.yticks(fontsize=13)
plt.show()

Y para terminar esta parte he sacado la tabla de correlación de todas las variables numericas

He verificado con el coeficiente de correlación de Pearson que la hipotesis estaba en lo correcto

A la función del 'grafico de correlacion con dispersion' le he añadido los nombres de las variables en el gráfico para una mejor comprensión.

In [ ]:
    plt.xlabel(columna_x)
    plt.ylabel(columna_y)
    for i in range(len(df)):
        equipo = df['Equipo'].iloc[i]
        plt.annotate(equipo, (df[columna_x].iloc[i], df[columna_y].iloc[i]),
                     textcoords='offset points', xytext=(0, 3), ha='center')

He sacado la matriz de correlación (heatmap) para los tres dataframe, los cuales me aportaron muchisima información y claridad. Además del Pairplot.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(matriz20, annot=True, fmt='.2f', cmap='coolwarm',
            cbar=True, square=True, linewidths=.5)

plt.title('Matriz de correlación')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.show()

He utilizado la función bubble_plot, aunque también la he modificado para introducirle mas variables. Le he puesto que los puntos puedan agrandarse dependiendo de alguna columna, en este caso los puntos, también le he puesto una linea de progresión media en los valores y como antes el nombre de los valores(equipos) a cada punto.

Tambien me he importado la librería mplsoccer para graficar un campo de fútbol, aunque esta la he guardado para usarla en la portada de la presentación.

In [ ]:
from mplsoccer.pitch import Pitch

pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
fig, ax = pitch.draw()
plt.savefig('campo.png')

Por útlimo he creado dos nuevos gráficos, el primero uno simple de barras, para mostrar el PV y su aumento del mínimo a lo largo de las temporadas que se podrá ver también en la presentación. Y para terminar un gráfico de barras pero con una linea central entre el negativo y el positivo para poder mostrar la diferencia entre la posesión y el valor de la posesión que también se verá en la presentación.

In [ ]:
df_liga22_sorted = df_liga22.sort_values('Clasificacion')

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x='Equipo', y='PV', data=df_liga22_sorted)
plt.xlabel('Equipos')
plt.ylabel('PV')
plt.title('PV de Equipos en la Liga Española (Ordenado por Clasificación)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()  
plt.savefig('posesion_liga22.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colores = ['red' if valor < 0 else 'blue' for valor in df_liga20['Diferencia posesion']]

bars = ax.bar(df_liga20.Equipo, df_liga20['Diferencia posesion'], color=colores)

ax.axhline(y=0, color='black', linestyle='--', linewidth=1)

ax.set_ylabel('Valores')
ax.set_title('Diferencia entre posesión y PV')


for bar, valor in zip(bars, df_liga20['Diferencia posesion']):
    height = bar.get_height()
    ax.annotate(f'{valor:.1f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3) if valor >= 0 else (0, -15),
                textcoords='offset points', ha='center', va='bottom' if valor >= 0 else 'top')
plt.xticks(rotation=45)
plt.savefig('diferencia_liga20.png')
plt.show()